<a href="https://colab.research.google.com/github/juancamposg4s/automatizacion_transportesMQ/blob/main/Transportes_AutomatizadosMJC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Organizador de Planillas de Transportes G4S**

El siguiente programa ayuda a organizar las **planillas de transportes y horas extra exclusivas de G4S** que están en los archivos de **Google Sheets** que a su vez están subidos a **Google Drive**. Este ayuda a filtrar cada hoja, ocultando elementos no deseados y agregando los que son necesarios, mejorando visualmente su formato para que sea más fácil la gestión por parte del usuario.

## **Funciones del Programa**

Lo que realizará el programa, en primer lugar, es verificar si la planilla de transportes o la tabla de horas extras están diligenciadas o no, a base de esto podrá realizar sus otras funciones en el siguiente orden:
1. Arreglar la formula para las celdas de la hoja 'GENERAL' con el fin de evitar registros erróneos.
2. Si la planilla de transportes está diligenciada, se organizará de la siguiente manera:
  - Cambiar el área de "G4S TECHNOLOGY COLOMBIA S.A." a "G4S INFOTEC" si algún registro pertenece a Colsubsidio, Compensar, Corona o Vanti.
  - Aplicar el formato de fecha correcto a los registros: Tipo de letra "Arial", tamaño de letra 10 y formato de fecha d mm yyy, que se traduce a 25 nov 2024, por ejemplo.
  - Aplicar correctamente la formula a la columna del Nit de la empresa de transportes, con el fin de que no hayan registros sin el Nit.
3. Si tanto la planilla de transportes como la tabla de horas extras están vacías, sin diligenciar, la hoja se ocultará.
4. Si la tabla de horas extra está vacía, se ocultarán las filas en las que se encuentra dicha tabla.
5. Si únicamente la planilla de transportes está vacía, se ocultarán las filas '1:103' que es donde se encuentra la planilla, además de eso la hoja cambiará a color rojo.
6. Si ninguna, de las planillas o tabla de extras, está vacía, se cambiará la hoja a color rojo.

## **Modo de Uso**

**Lo primero que debe tener en cuenta es que este código solo puede ser ejecutado en el entorno virtual de Google Colab, de lo contrario puede que no funcione como debería.**

Al momento en que quiera organizar sus planillas, lo primero que deberá hacer es dirigirse a su archivo en Google Drive y copiar el enlace de acceso al mismo, esto lo podrá hacer dirigiéndose a la sección "Compartir" de su archivo de Google Sheets y presionando el botón "Copiar enlace".

Una vez tenga el enlace al archivo, puede ejecutar el código, lo podrá hacer presionando el botón de Play que se encuentra ubicado a mano izquierda del bloque de código. Hecho este paso pasará lo siguiente:

1. El código empezará a ejecutarse, podrá darse cuenta de esto dirigiéndose a la parte de abajo del código en donde saldrá el Output del mismo, allí lo primero que se hará es instalar las librerías necesarias para ejecutar el código.

2. Google Colab solicitará permiso a su cuenta de Gmail para poder editar archivos de Google Sheets, deberá aceptar y dar permiso, de lo contrario el código no podrá hacer su trabajo.

3. El programa le solicitará la URL o enlace del archivo a procesar (de ahí la importancia de que previamente haya copiado el enlace al archivo), deberá pegarlo en el Output del programa y presionar la tecla Enter para que se empiece a procesar.
  - Ejemplo:
    ```
    Ingresa la URL del archivo a procesar: https://docs.google.com/spreadsheets/d/1OULey9EVdwfJzIYM1n4Ah79MorAvFNwUs8TRW-UzqG8/edit?usp=sharing
    Procesando archivo...
    Esto puede tomar unos minutos
    ```

  Una vez que haya hecho esto, el archivo empezará a procesarse, aparecerá una barra de progreso que indica el estado de la ejecución. El proceso puede tardar unos minutos, dependiendo de la cantidad de hojas en el archivo, pero una vez haya finalizado podrá dirigirse a su archivo y **verificar los cambios realizados**.


## **Documentación de las Librerías Usadas**

La librerías implementadas en este programa fueron las siguientes:
- [API de Python para Google Sheets: gspread](https://docs.gspread.org/en/v6.1.3/)

- [Servicio de Cuentas de Google: google.oauth2.service_account](https://google-auth.readthedocs.io/en/master/reference/google.oauth2.service_account.html)

- [Librería de Google Colab para Autenticación: google.colab.auth](https://github.com/googlecolab/colabtools/tree/main/google/colab)

- [Librería de Autenticación de Google: google.auth.default](https://google-auth.readthedocs.io/en/master/reference/google.auth.html#google.auth.default)

- [Librería para agregar barra de progreso: tqdm](https://tqdm.github.io/)

In [ ]:
import gspread #API para Google Sheets
from google.oauth2.service_account import Credentials #Servicio de cuentas de Google
from google.colab import auth # Autenticacion de Google Colab
from google.auth import default # Autenticacion de Google
import time
!pip install tqdm --upgrade  # Instalar tqdm si no esta instalado
from tqdm.notebook import tqdm #Para agrega la barra de progreso

# Autenticarse en Google Drive
auth.authenticate_user()
creds, _ = default()

# Crear una instancia del cliente de Google Sheets con la autenticación
gc = gspread.authorize(creds)

# URL del archivo de Google Sheets
spreadsheet_url = input("Ingresa la URL del archivo a procesar 👉🏼 ")

# Abrir el archivo de Google Sheets
try:
  shfile = gc.open_by_url(spreadsheet_url)
except Exception as e:
  raise Exception(f"ಠಿ_ಠ Hubo un error con la URL ingresada. Por favor, verifica que sea correcta.")

cell_range_trans = 'D11:F19' #Rango de celdas a verificar en la planilla transportes
cell_range_ex = 'E109:K111' #Rango de celdas a verificar en la tabla de extras
print('Procesando archivo...\nEsto puede tomar unos minutos 😴')
total_sheets = len(shfile.worksheets())

infotec = {} # Diccionaio para guardar empleados de INFOTEC
completed_sheets = {} #Contador para las  hojas diligenciadas
num_extras = 0 #Contador para las horas extra
# Iterar sobre cada hoja en el archivo
for worksheet in tqdm(shfile.worksheets(), total=total_sheets, desc="Procesando hojas"):

  if worksheet.title.isnumeric():
    rowextras = worksheet.findall("CEDULA", in_column=1) # Ayuda a saber el rango en el que se encuentra la tabla de extras
    endrowextras = worksheet.findall("Total ", in_column=1) # Ayuda a saber el rango en el que se encuentra la tabla de extras

    try:
      cell_range_ex = f'E{int(rowextras[0].row)+3}:K{int(rowextras[0].row)+6}' #Rango de celdas a verificar en la tabla de extras
    except Exception as e:
      print(f"X﹏X Error al verificar extras en la hoja '{worksheet.title}': {e}")

    # Obener valores en los rangos de celdas dados
    cell_values_trans = worksheet.range(cell_range_trans)
    cell_values_ex = worksheet.range(cell_range_ex)
    # Verificar si el rango establecido está vacío
    trans_is_empty = all(cell.value == '' for cell in cell_values_trans)
    ex_is_empty = all(cell.value == '' for cell in cell_values_ex)

  elif worksheet.title == 'GENERAL': # Cuando llega a la hoja 'GENERAL'
    try:
      # Obtener todos los valores de la columna A
      cedulas = worksheet.col_values(1)
      cells_to_update = []
      for row, cedula in enumerate(cedulas, start=1): # Iterar sobre cada fila
          cedula = cedula.replace('.', '')
          if cedula.isnumeric():
            formulas = [
                f"=(SUMAR.SI.CONJUNTO('{cedula}'!$L$11:$L$29;'{cedula}'!$J$11:$J$29;GENERAL!${col}$2))"
                for col in ('E', 'F', 'G', 'H')
            ]
            # Añadir todas las celdas con fórmulas a la lista
            cells_to_update.extend([gspread.Cell(row, col_index + 5, formula) for col_index, formula in enumerate(formulas)]) # Se suma 5 para que concuerde con las columnas E, F, G y H
      worksheet.update_cells(cells_to_update, value_input_option='USER_ENTERED') # Aplicar la formula
      print("Registros de la hoja 'GENERAL' mejorados 😼")
    except Exception as e:
      print(f"X﹏X Error al procesar la hoja 'GENERAL': {e}")
    finally:
      continue
  else:
    continue

  #Verifica si la planilla de transportes esta diligenciada
  if not trans_is_empty:
    employee_name = str(worksheet.acell('C5').value).strip() #Obtener el nombre del empleado
    completed_sheets[employee_name] = worksheet.title
    # Almacena las celdas a actualizar
    cells_to_update = [
        gspread.Cell(row, 11, f'=SI.ERROR(BUSCARV(J{row};$J$38:$K$48;2;0);"")')
        for row in range(11, 30)
    ]

    # Verificar si los transportes pertenecen a Infotec
    for cell in cell_values_trans:
      for word in ['colsubsidio', 'compensar', 'corona', 'vanti', 'halliburton']: # Clientes por verificar
        if word in cell.value.lower():
          cells_to_update.append(gspread.Cell(1, 4, 'G4S INFOTEC')) # Actualizar el encabezado de la planilla
          worksheet.update_tab_color('#0000ff')
          infotec[str(worksheet.acell('C5').value)] = word # Agregar al diccionario el nombre del empleado y cliente
          break

    #Organizar las celdas de fechas
    worksheet.format('B11:B29', {
        "numberFormat": {
              "type": 'DATE',
              "pattern": 'd mmm yyy'
            },
        "textFormat": {
            "fontFamily": 'Arial',
            "fontSize": 10,
            }
    })
    #Aplicar formula a las celdas
    worksheet.update_cells(cells_to_update, value_input_option='USER_ENTERED')

  if trans_is_empty and ex_is_empty:
    # Ocultar la planilla de transportes y la tabla de extras si estan vacías
    worksheet.hide()
    time.sleep(0.5) #Esperar antes de la siguiente llamada a la API
  elif ex_is_empty:
    #Ocultar tabla de extras si está vacia
    try:
      worksheet.hide_rows(int(rowextras[0].row)-2, int(endrowextras[0].row)+2)
      time.sleep(0.5) #Esperar antes de la siguiente llamada a la API
    except Exception as e:
      print(f"X﹏X Error al ocultar la tabla de horas extra en la hoja {worksheet.title}: {e}")
  elif trans_is_empty:
    #Ocultar planilla de transportes si esta vacia y cambiar color de la hoja a rojo
    try:
      worksheet.hide_rows(0, int(rowextras[0].row)-2)
    except Exception as e:
      print(f"X﹏X Error al ocultar la planilla de transportes en la hoja {worksheet.title}: {e}")
    num_extras += 1
    worksheet.update_tab_color('#ff0000')
    time.sleep(0.3) #Esperar antes de la siguiente llamada a la API
  else:
    #Cambiar color de la hoja a rojo si ninguno esta vacio
    worksheet.update_tab_color('#ff0000')
    num_extras += 1
    time.sleep(0.3) #Esperar antes de la siguiente llamada a la API

print(f"\nTotal de tablas de horas extra diligenciadas: {num_extras}")
print(f"\nTotal de planillas de transportes diligenciadas: {len(completed_sheets)}")
for tecnico, cedula in completed_sheets.items():
  print(f"  {tecnico}: {cedula}")

print(f"\nTotal técnicos pertenecientes a INFOTEC: {len(infotec)}")
for tecnico, cliente in infotec.items():
  print(f"  {tecnico}: {cliente}")

print("\nCiclo de ejecución finalizado, por favor verifique los cambios realizados en su archivo. 😸")

Ingresa la URL del archivo a procesar 👉🏼 https://docs.google.com/spreadsheets/d/1QhaFmPg06NGAUztxAO4Aj0ea65sgyJwLievHKqydI3U/edit?usp=sharing
Procesando archivo...
Esto puede tomar unos minutos 😴


Procesando hojas:   0%|          | 0/84 [00:00<?, ?it/s]

X﹏X Error al procesar la hoja 'GENERAL': {'code': 400, 'message': 'Estás intentando editar una celda o un objeto protegidos. Ponte en contacto con el propietario de la hoja de cálculo para desprotegerla si es necesario modificarla.', 'status': 'INVALID_ARGUMENT'}
X﹏X Error al verificar extras en la hoja '80172289': list index out of range
X﹏X Error al ocultar la tabla de horas extra en la hoja 80172289: list index out of range

Total de tablas de horas extra diligenciadas: 5

Total de planillas de transportes diligenciadas: 25
  Andrea Selene Osorio Canasto: 1000179099
  María Alejandra Peraza Medina: 1001060680
  jhan Carlos brochero gil: 1004149141
  CRISTIAN CAMILO MOYA RUIZ: 1000254039
  Rafael Castro Carballo: 1083031952
  ian Esteban cortes pimiento: 1021392248
  OSCAR JAVIER SIERRA CHUNZA: 1000366202
  CRISTIAN DAVID CASTELLANOS GARCIA: 1000514306
  NATAN DAVID PEDRAZA RODRIGUEZ: 1001216102
  DAYRON STEVENS GOMEZ RODRIGUEZ: 1007356109
  SERGIO ANDRES BOHORQUEZ CARO: 1030623105
 

# **Generador de Archivos PDF para las Planillas de Transporte**

El siguiente programa ayuda a generar archivos PDF a las **planillas de transportes y horas extra exclusivas de G4S** que están en los archivos de **Google Sheets** que a su vez están subidos a **Google Drive**.

## **Funciones del Programa**

Lo que realizará el programa, en primer lugar, es verificar si la planilla de transportes está diligenciada o no, a base de esto generará un archivo PDF con las siguientes especificaciones:
1. Tamaño del papel: carta
2. Orientación: vertical
3. Escala: Ajustar al ancho
4. No mostrar el nombre de la hoja, ni el título, ni el número de página, ni líneas de cuadrícula.
5. Margen superior e inferior: 0.75 pulgadas, correspondientes a 1.905 cm
6. Margen derecho e izquierdo: 0.25 pulgadas, correspondientes a 0.635 cm
7. Imprimir solo el rango de celdas 'A1:L46', correspondiente solo a la planilla de transportes.

Este formato se aplica para cada una de las hojas que contengan planillas diligenciadas, despues de generar el PDF para cada una lo almacenará en un archivo **.zip** que se descargará en su ordenador.

## **Modo de Uso**

**Lo primero que debe tener en cuenta es que este código solo puede ser ejecutado en el entorno virtual de Google Colab, de lo contrario puede que no funcione como debería.**

Al momento en que quiera generar el PDF de sus planillas, lo primero que deberá hacer es dirigirse a su archivo en Google Drive y copiar el enlace de acceso al mismo, esto lo podrá hacer dirigiéndose a la sección "Compartir" de su archivo de Google Sheets y presionando el botón "Copiar enlace".

Una vez tenga el enlace al archivo, puede ejecutar el código, lo podrá hacer presionando el botón de Play que se encuentra ubicado a mano izquierda del bloque de código. Hecho este paso pasará lo siguiente:

1. El código empezará a ejecutarse, podrá darse cuenta de esto dirigiéndose a la parte de abajo del código en donde saldrá el Output del mismo, allí lo primero que se hará es instalar las librerías necesarias para ejecutar el código.

2. Google Colab solicitará permiso a su cuenta de Gmail para poder editar archivos de Google Sheets, deberá aceptar y dar permiso, de lo contrario el código no podrá hacer su trabajo.

3. El programa le solicitará la URL o enlace del archivo a procesar (de ahí la importancia de que previamente haya copiado el enlace al archivo), deberá pegarlo en el Output del programa y presionar la tecla Enter para que se empiece a procesar.
  - Ejemplo:
    ```
    Ingresa la URL del archivo a procesar: https://docs.google.com/spreadsheets/d/1OULey9EVdwfJzIYM1n4Ah79MorAvFNwUs8TRW-UzqG8/edit?usp=sharing
    Procesando archivo...
    Esto puede tomar unos minutos
    ```

  Una vez que haya hecho esto, el archivo empezará a procesarse, aparecerá una barra de progreso que indica el estado de la ejecución. El proceso puede tardar unos minutos, dependiendo de la cantidad de hojas en el archivo, pero una vez haya finalizado se descargará automáticamente en su ordenador el archivo **.zip** que contiene los archivos PDF.


## **Documentación de las Librerías Usadas**

La librerías implementadas en este programa fueron las siguientes:
- [API de Python para Google Sheets: gspread](https://docs.gspread.org/en/v6.1.3/)

- [Servicio de Cuentas de Google: google.oauth2.service_account](https://google-auth.readthedocs.io/en/master/reference/google.oauth2.service_account.html)

- [Librería de Google Colab para Autenticación: google.colab.auth](https://github.com/googlecolab/colabtools/tree/main/google/colab)

- [Librería de Autenticación de Google: google.auth.default](https://google-auth.readthedocs.io/en/master/reference/google.auth.html#google.auth.default)

- [Librería para solicitudes HTTP: requests](https://realpython.com/python-requests/)

- [Librería para generar alchivo .zip: zipfile](https://docs.python.org/3/library/zipfile.html#zipfile.ZipFile)

- [Librería de Google Colab para gestionar archivos: google.colab.files](https://github.com/googlecolab/colabtools/blob/main/google/colab/files.py)

- [Librería para agregar barra de progreso: tqdm](https://tqdm.github.io/)

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
from google.colab import auth
from google.auth import default
import requests
import zipfile
from google.colab import files
import time
!pip install tqdm --upgrade  # Instalar tqdm si no esta instalado
from tqdm.notebook import tqdm #Para agrega la barra de progreso

# Autenticarse en Google Drive
auth.authenticate_user()
creds, _ = default()

# Crear una instancia del cliente de Google Sheets con la autenticación
gc = gspread.authorize(creds)

# URL del archivo de Google Sheets
spreadsheet_url = input("Ingresa la URL del archivo a procesar 👉🏼 ")

# Abrir el archivo de Google Sheets
try:
  shfile = gc.open_by_url(spreadsheet_url)
except:
  raise Exception("ಠಿ_ಠ Hubo un error con la URL ingresada. Por favor, verifica que sea correcta.")

# Obtener todas las hojas del archivo de Google Sheets
worksheets = shfile.worksheets()
# Obtener el ID del archivo de Google Sheets
spreadsheet_id = shfile.id
print('Procesando archivo...\nEsto puede tomar unos minutos 😴')

# Función para exportar una hoja como PDF
def export_sheet_as_pdf(spreadsheet_id, worksheet_id, pdf_name):
  # URL de la API de Google Sheets para exportar una hoja como PDF
  url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=pdf&gid={worksheet_id}"

  # Configurar parámetros para la exportación
  params = {
      'size': 'Letter',  # Tamaño del papel
      'portrait': 'true',  # Orientación vertical
      'fitw': 'true',  # Ajustar a ancho
      'sheetnames': 'false',  # No mostrar nombres de hoja
      'printtitle': 'false',  # No mostrar título
      'pagenumbers': 'false',  # No mostrar números de página
      'gridlines': 'false',  # No mostrar líneas de cuadrícula
      'fzr': 'false',  # No congelar filas
      'zoom': 100, # Ajustar el zoom al 100%
      'range': 'A1:L46',
      'top_margin': '0.75', # Tamaño en pulgadas, correspondiente a 1.905 cm
      'bottom_margin': '0.75', # Tamaño en pulgadas, correspondiente a 1.905 cm
      'left_margin': '0.25', # Tamaño en pulgadas, correspondiente a 0.635 cm
      'right_margin': '0.25', # Tamaño en pulgadas, correspondiente a 0.635 cm
  }

  for attemp in range(1, 4):
    # Hacer la solicitud para exportar el PDF
    response = requests.get(url, params=params, headers={'Authorization': f'Bearer {creds.token}'})

    if response.status_code == 200:
      with open(pdf_name, 'wb') as f:
        f.write(response.content)
      print(f"Archivo PDF creado: {pdf_name}")
      return pdf_name  # Agregar el nombre del archivo PDF a la lista
    else:
      time.sleep(attemp * 3) #Debe estar en 3 para que se generen todos los archivos
  print(f"No se creó el archivo {pdf_name} después de varios intentos: Error {response.status_code}")
  return None

pdf_files = []
not_created = []
sheets_to_export = []
# Iterar sobre cada hoja en el archivo
for worksheet in tqdm(worksheets, total=len(worksheets), desc="Obteniendo hojas para exportar"):
  if worksheet.title.isnumeric():
    # Obtener los valores del rango de celdas y el valor de la celda C5 en una sola llamada
    cell_values_trans = worksheet.batch_get(['D11:E13', 'C5'])
    trans_range = cell_values_trans[0]
    # Verificar si el rango establecido está vacío
    trans_is_empty = all(cell == '' for row in trans_range for cell in row)

    if not trans_is_empty:
      try:
        employee_name = cell_values_trans[1][0][0].upper().strip() + '.pdf' # Valor de C5
      except Exception:
        employee_name = worksheet.title + '.pdf' # Si la celda C5 está vacia, se toma el nombre de la hoja
      sheets_to_export.append((worksheet.id, employee_name))

print("Hojas obtenidas, iniciando exportación...")

for worksheet_id, employee_name in tqdm(sheets_to_export, total=len(sheets_to_export), desc="Exportando hojas"):
  pdf = export_sheet_as_pdf(spreadsheet_id, worksheet_id, employee_name)
  if pdf:
    pdf_files.append(pdf)
    time.sleep(1)
  else:
    not_created.append((worksheet_id, employee_name))

if not_created:
  print("\n(っ °Д °;)っ¡Hubo archivos que no fueron creados!")
  time.sleep(2)
  for worksheetID, pdf_name in tqdm(not_created, desc="Intentando exportar nuevamente"):
    pdf = export_sheet_as_pdf(spreadsheet_id, worksheetID, pdf_name)
    if pdf:
      pdf_files.append(pdf)
      not_created.remove((worksheetID, pdf_name))

    # Crear un archivo ZIP con todos los PDFs
zip_filename = f"pdfs_{shfile.title.strip()}.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for pdf_file in pdf_files:
        zipf.write(pdf_file)  # Escribir el archivo PDF en el archivo ZIP

# Imprimir lista de los archivos PDF no generados por algún error
if not_created:
  print("\n( ´･･)ﾉ(._.`) No fue posible generar PDF para las hojas correspondientes a:")
  for id, pdf in not_created:
    print(pdf)

print(f"\n{len(pdf_files)}/{len(sheets_to_export)} archivos PDF generados, se iniciará la descarga de los mismos en un archivo .zip.")
# Descargar el archivo ZIP
files.download(zip_filename)
print(f"Archivo {zip_filename} descargado 😸")

Ingresa la URL del archivo a procesar 👉🏼 https://docs.google.com/spreadsheets/d/1QhaFmPg06NGAUztxAO4Aj0ea65sgyJwLievHKqydI3U/edit?usp=sharing
Procesando archivo...
Esto puede tomar unos minutos 😴


Obteniendo hojas para exportar:   0%|          | 0/84 [00:00<?, ?it/s]

Hojas obtenidas, iniciando exportación...


Exportando hojas:   0%|          | 0/24 [00:00<?, ?it/s]

Archivo PDF creado: ANDREA SELENE OSORIO CANASTO.pdf
Archivo PDF creado: MARÍA ALEJANDRA PERAZA MEDINA.pdf
Archivo PDF creado: JHAN CARLOS BROCHERO GIL.pdf
Archivo PDF creado: CRISTIAN CAMILO MOYA RUIZ.pdf
Archivo PDF creado: RAFAEL CASTRO CARBALLO.pdf
Archivo PDF creado: IAN ESTEBAN CORTES PIMIENTO.pdf
Archivo PDF creado: OSCAR JAVIER SIERRA CHUNZA.pdf
Archivo PDF creado: CRISTIAN DAVID CASTELLANOS GARCIA.pdf
Archivo PDF creado: NATAN DAVID PEDRAZA RODRIGUEZ.pdf
Archivo PDF creado: DAYRON STEVENS GOMEZ RODRIGUEZ.pdf
Archivo PDF creado: SERGIO ANDRES BOHORQUEZ CARO.pdf
Archivo PDF creado: JOHAN SNNEIDER ESCOBAR HERNANDEZ.pdf
Archivo PDF creado: JUAN ANDRES GUATAMA RODRIGUEZ.pdf
Archivo PDF creado: JONATHAN DAVID SIERRA VARGAS.pdf
Archivo PDF creado: HAROLD MATEO CAÑON VILLAMIL.pdf
Archivo PDF creado: JUAN JOSE CONTRERAS MOJICA.pdf
Archivo PDF creado: DANIEL STIVEN ORTEGA CRUZ.pdf
Archivo PDF creado: JOHN GEYVER BARON GOMEZ.pdf
Archivo PDF creado: JUAN GABRIEL CALDERÓN RAMOS.pdf
Archivo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivo pdfs_16 - 21 Diciembre Transportes - Extras.zip descargado 😸
